In [4]:
import pandas as pd
import time
import io
from api_request_funtions import ApiGet
from bq_transfers import BqDataTransfers 
# from main import fetch_and_save

In [5]:
thrivecart_get = ApiGet(http='thrivecart.com', api_key='TZ5TJYBR-FDB85IBI-0RFTB00N-VQ7ZFY2S')
thrivecart_save = BqDataTransfers(gcp_project_id= 'arboreal-cat-451816-n0', bq_data_set = 'thrive_cart')
bq_client = thrivecart_save.get_bq_client("/Users/shami/Library/Mobile Documents/com~apple~CloudDocs/Personal Projects/vexis/vexis_bq_writter.json")

In [6]:
# def fetch_transactions(event, context):
x = thrivecart_get.fetch_data(end_point=f'/api/external/transactions?page=0&perPage=0&query=&transactionType=any')
# pages = list(range(1, round(x['meta']['total'] / 100) + 1))
pages = list(range(1, 5))  #limiting to 5 pages for now
for page in pages:
    x = thrivecart_get.fetch_data(
        end_point=f'/api/external/transactions?page={page}&perPage=100&query=&transactionType=any')
    ndjson_lines = "\n".join(json.dumps(transaction) for transaction in x["transactions"])
    ndjson_buffer = io.StringIO(ndjson_lines)
    thrivecart_save.start_transfer_json(bq_client=bq_client, file=ndjson_buffer,
                                          destination_table = 'transactions', write_options='append')
    time.sleep(0.5)

KeyError: 'transactions'

In [7]:
x

{'error': 'api.ratelimited',
 'reason': 'Your request has been rate limited; please slow down and try again shortly.'}